<a href="https://colab.research.google.com/github/KhannaShivang/RecipeRecomendation/blob/main/Recipe_Recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install fuzzywuzzy[speedup]

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Indian_Food_Dataset.csv')
df.head()

,RecipeName,Ingredients,TotalTimeInMins,Cuisine,Instructions,URL,Ingredients.1,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12


In [ ]:
def clean_ingredients(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    return ' '.join([synonym_dict.get(word, word) for word in words])

df['cleaned_ingredients'] = df['Ingredients.1'].apply(clean_ingredients)

# Vectorize cleaned ingredients
vectorizer = TfidfVectorizer()
ingredient_matrix = vectorizer.fit_transform(df['cleaned_ingredients'])
print("Ingredient matrix shape :",ingredient_matrix.shape)

Ingredient matrix shape : (5938, 18307)


In [ ]:
# Define a dictionary for synonyms
synonym_dict = {
    'wheet': 'wheat',
    'spinich': 'spinach',
    'coriander': 'cilantro',
    'rajma': 'kidney beans'
    # Add more synonyms here as needed
}

# Map synonyms in the cleaned ingredients
def map_synonyms(text):
    words = text.split()
    return ' '.join([synonym_dict.get(word, word) for word in words])

df['cleaned_ingredients'] = df['cleaned_ingredients'].apply(map_synonyms)


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Define and fit the KNN model with Euclidean distance
model = NearestNeighbors(n_neighbors=3, metric='euclidean')
model.fit(ingredient_matrix)

def recommend_recipes_knn(input_features):
    # Clean input ingredients
    cleaned_input = clean_ingredients(' '.join(input_features))

    # Vectorize the cleaned input ingredients
    input_vector = vectorizer.transform([cleaned_input])

    # Get the indices of the top 3 similar recipes using KNN
    distances, indices = model.kneighbors(input_vector, n_neighbors=3)  # Using the defined model

    # Retrieve recommendations
    recommendations = df.iloc[indices[0]]

    # Debug: Check if recommendations DataFrame is correctly populated
    print("Initial Recommendations:")
    print(recommendations[['RecipeName', 'Ingredients.1']])  # Displaying only the names and ingredients

    # Filter recommendations to include only those with at least 2 of the core ingredients
    def contains_core_ingredients(recipe_ingredients, input_ingredients):
        recipe_ingredients = clean_ingredients(recipe_ingredients).split()
        match_count = sum(1 for ingredient in input_ingredients if ingredient in recipe_ingredients)
        return match_count >= 2  # You can adjust this threshold

    # Apply filtering
    filtered_recommendations = recommendations[recommendations['Ingredients.1'].apply(lambda x: contains_core_ingredients(x, input_features))]

    # Debug: Check if any recipes are filtered correctly
    print("\nFiltered Recommendations:")
    print(filtered_recommendations[['RecipeName', 'Ingredients.1']])  # Display filtered results

    return filtered_recommendations[['RecipeName', 'TotalTimeInMins', 'Ingredients.1', 'Instructions', 'image-url']]


In [ ]:
df.columns

Index(['RecipeName', 'Ingredients', 'TotalTimeInMins', 'Cuisine',
       'Instructions', 'URL', 'Ingredients.1', 'image-url', 'Ingredient-count',
       'cleaned_ingredients'],
      dtype='object')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def recommend_recipes(input_features):
    # Clean input ingredients
    cleaned_input = clean_ingredients(' '.join(input_features))

    # Vectorize the cleaned input ingredients
    input_vector = vectorizer.transform([cleaned_input])

    # Weighting the input ingredients based on their position
    weights = [len(input_features) - i for i in range(len(input_features))]

    # Initialize a weighted input vector (same shape as the vectorized input)
    weighted_input_vector = input_vector.copy()

    # Apply the weights to the individual words' TF-IDF values
    for i, weight in enumerate(weights):
        weighted_input_vector[0, i] *= weight

    # Calculate similarity with weighted input vector
    similarity = cosine_similarity(weighted_input_vector, ingredient_matrix)

    # Get the indices of the top 5 recipes based on similarity
    indices = similarity.argsort()[0][-5:][::-1]

    # Retrieve recommendations
    recommendations = df.iloc[indices]

    # Filter recommendations to include those with core ingredients
    filtered_rec = recommendations[recommendations['cleaned_ingredients'].str.contains('|'.join(input_features))]

    # If no filtered results, return top 5 anyway
    if filtered_rec.empty:
        return recommendations[['RecipeName', 'TotalTimeInMins', 'Ingredients.1', 'Instructions', 'image-url']]

    return filtered_rec[['RecipeName', 'TotalTimeInMins', 'Ingredients.1', 'Instructions', 'image-url']]


In [ ]:
# Sample input features
input_features = ['rajma', 'bell pepper', 'mushroom', 'zucchini', 'black pepper', 'soy sauce']


# Get recommendations
recommendations = recommend_recipes_knn(input_features)
recommendations


Initial Recommendations:
                                        RecipeName  \
4288               Schezwan Chicken Sizzler Recipe   
4543  Chilli Ginger Vegetable Tofu Stir Fry Recipe   
77             Crunchy Ginger Capsicum Rice Recipe   

                                          Ingredients.1  
4288  yellow bell pepper (capsicum),spring onion gre...  
4543  yellow bell pepper (capsicum),coriander (dhani...  
77    yellow bell pepper (capsicum),basmati rice,red...  

Filtered Recommendations:
Empty DataFrame
Columns: [RecipeName, Ingredients.1]
Index: []


,RecipeName,TotalTimeInMins,Ingredients.1,Instructions,image-url
